In [54]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [55]:
cd 'drive'/'MyDrive'/'Dacon'/'LDI'

[Errno 2] No such file or directory: 'drive/MyDrive/Dacon/LDI'
/content/drive/MyDrive/Dacon/LDI


In [56]:
train_err_data = pd.read_csv('./dataset/train_err_data.csv',thousands = ',')
train_quality_data = pd.read_csv('./dataset/train_quality_data.csv',thousands = ',')
train_problem_data = pd.read_csv('./dataset/train_problem_data.csv',thousands = ',')

In [57]:
df_tqd = train_quality_data.drop_duplicates()
df_ted = train_err_data.drop_duplicates()
df_tpd = train_problem_data.drop_duplicates()

In [58]:
df_ted.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [59]:
df_ted['errtype'].value_counts()

23    2276011
31    1985037
22    1863495
15    1562517
4     1291986
16    1248142
32    1130247
40     730283
5      630280
26     530998
12     318847
11     305205
14     239212
28     215338
27     210131
33     178271
34     114833
41      99368
10      68832
7       51972
6       50433
17      45923
24      39064
42      34297
3       30497
2       26099
1       21513
39      15819
13      12123
35      10163
37       9336
36       9317
38       4539
18       2671
20       2080
25       1184
19        664
21        607
30        402
8         157
9         109
Name: errtype, dtype: int64

In [60]:
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'connection timeout'),'errtype'] = '23_ct'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'active'),'errtype'] = '23_as'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'standby'),'errtype'] = '23_ac'
df_ted.loc[(df_ted['errtype'] == 31) & (df_ted['errcode'] == '0'),'errtype'] = '31_0'
df_ted.loc[(df_ted['errtype'] == 40) & (df_ted['errcode'] == '0'),'errtype'] = '40_0'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [61]:
encodedfwvlst = ['04.16.3553','04.22.1750','04.33.1261','05.15.2138','04.22.1778',  # Encoding ted by fwversion
                 '04.33.1185','03.11.1167']
lst_fwv = list(df_ted['fwver'])
for i in range(len(lst_fwv)):
    if lst_fwv[i] not in encodedfwvlst:
        lst_fwv[i] = 'nope'
df_ted['fwver'] = lst_fwv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [62]:
encodedfwvlst = ['04.16.3553','04.22.1750','04.33.1261','05.15.2138','04.22.1778' # Encoding tqd by fwversion
                 '04.33.1185','03.11.1167']
lst_fwv = list(df_tqd['fwver'])
for i in range(len(lst_fwv)):
    if lst_fwv[i] not in encodedfwvlst:
        lst_fwv[i] = 'nope'
df_tqd['fwver'] = lst_fwv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [63]:
df_ted['fwver'].value_counts()

04.16.3553    5145420
04.22.1750    2748374
04.33.1261    2375758
05.15.2138    1841272
04.22.1778    1243732
04.33.1185     911660
03.11.1167     845808
nope           255978
Name: fwver, dtype: int64

In [64]:
for i in range(0,13):
    qstr = 'quality_'+str(i)
    b = df_tqd.loc[df_tqd[qstr] != -1,qstr] = 0

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [65]:
df_ted = df_ted.drop('fwver',axis=1)
df_ted = df_ted.drop('model_nm',axis=1)
df_tqd = df_tqd.drop('fwver',axis=1)
df_ted = df_ted.drop('errcode',axis=1)

In [66]:
df_encoded_ted = pd.get_dummies(df_ted,columns=['errtype'])

In [67]:
df_encoded_ted

,user_id,time,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,errtype_23_ac,errtype_23_as,errtype_23_ct,errtype_31_0,errtype_40_0
0,10000,20201101025616,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10000,20201101030309,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10000,20201101030309,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10000,20201101050514,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10000,20201101050515,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16554658,24999,20201130163051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16554659,24999,20201130172625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16554660,24999,20201130172625,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16554661,24999,20201130172631,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
lst_column = ['errtype_1', 'errtype_2', 'errtype_3', 'errtype_4', 'errtype_5', 'errtype_6', 'errtype_7', 'errtype_8', 'errtype_9', 'errtype_10', 'errtype_11', 'errtype_12', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_16', 'errtype_17', 'errtype_18', 'errtype_19', 'errtype_20', 'errtype_21', 'errtype_22', 'errtype_23', 'errtype_24', 'errtype_25', 'errtype_26', 'errtype_27', 'errtype_28', 'errtype_30', 'errtype_31', 'errtype_32', 'errtype_33', 'errtype_34', 'errtype_35', 'errtype_36', 'errtype_37', 'errtype_38', 'errtype_39', 'errtype_40', 'errtype_41', 'errtype_42', 'errtype_23_ac', 'errtype_23_as', 'errtype_23_ct', 'errtype_31_0', 'errtype_40_0', 'quality_0', 'quality_1', 'quality_2', 'quality_3', 'quality_4', 'quality_5', 'quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']# Make a output datafarme
df_x = pd.DataFrame(columns = lst_column)
df_x

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,errtype_23_ac,errtype_23_as,errtype_23_ct,errtype_31_0,errtype_40_0,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


In [70]:
noProblemUser = list(set(df_ted['user_id'].unique()) - set(df_tpd['user_id'].unique()))
i=1
for user_id in noProblemUser:
    print(str(i),"/",str(len(noProblemUser)),"finished.")
    i+=1
    df_check_id_ted = df_encoded_ted[df_encoded_ted.user_id == user_id]
    df_check_id_tqd = df_tqd[df_tqd.user_id == user_id]
    df_check_id_ted = df_check_id_ted.drop('time',axis = 1)
    df_temp_ted = df_check_id_ted.drop('user_id',axis = 1)
    df_check_id_tqd = df_check_id_tqd.drop('time',axis = 1)
    df_temp_tqd = df_check_id_tqd.drop('user_id',axis = 1)
    df_res = pd.concat([np.sum(df_temp_ted), np.sum(df_temp_tqd)])
    df_x = df_x.append(pd.Series(df_res,index = df_x.columns),ignore_index=True)
    


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5001 / 10000 finished.
5002 / 10000 finished.
5003 / 10000 finished.
5004 / 10000 finished.
5005 / 10000 finished.
5006 / 10000 finished.
5007 / 10000 finished.
5008 / 10000 finished.
5009 / 10000 finished.
5010 / 10000 finished.
5011 / 10000 finished.
5012 / 10000 finished.
5013 / 10000 finished.
5014 / 10000 finished.
5015 / 10000 finished.
5016 / 10000 finished.
5017 / 10000 finished.
5018 / 10000 finished.
5019 / 10000 finished.
5020 / 10000 finished.
5021 / 10000 finished.
5022 / 10000 finished.
5023 / 10000 finished.
5024 / 10000 finished.
5025 / 10000 finished.
5026 / 10000 finished.
5027 / 10000 finished.
5028 / 10000 finished.
5029 / 10000 finished.
5030 / 10000 finished.
5031 / 10000 finished.
5032 / 10000 finished.
5033 / 10000 finished.
5034 / 10000 finished.
5035 / 10000 finished.
5036 / 10000 finished.
5037 / 10000 finished.
5038 / 10000 finished.
5039 / 10000 finished.
5040 / 10000 finished.
5041 / 10000 finished.
5042 / 10000 finished

In [ ]:
df_x

In [71]:
import pickle
with open('noProblemList.txt', 'wb') as f:
    pickle.dump(df_x, f)